https://towardsdatascience.com/predicting-the-music-mood-of-a-song-with-deep-learning-c3ac2b45229e

https://realpython.com/python-nltk-sentiment-analysis/

MEDIUM : https://towardsdatascience.com/how-to-analyze-emotions-and-words-of-the-lyrics-from-your-favorite-music-artist-bbca10411283

In [2]:
#libraries used to extract, clean and manipulate the datafrom helpers import *
import pandas as pd
import numpy as np
import string#To plot the graphsfrom wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn')#library used to count the frequency of wordsfrom sklearn.feature_extraction.text import CountVectorizer#To create the sentiment analysis model, tokenization and lemmatizationimport nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
import nltk.data
nltk.download('vader_lexicon')
nltk.download('punkt')

import time

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Nmtur\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nmtur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

spotify_client_id = "3a2f65eb20164b24b59b7fdd6876bb51"
spotify_client_secret = "9f6d3a070cdf4eb8adda14a794d60469"

#Genius credentials
genius_token = "4SoLPv5UHyFZyAe5T-PhxS-bRCRrdNIgMNpeqjjIaWvnRbUmTpoGxd9knxBtlQyg"

#Authentication spotify - without user
client_credentials_manager = SpotifyClientCredentials(client_id="3a2f65eb20164b24b59b7fdd6876bb51", client_secret="9f6d3a070cdf4eb8adda14a794d60469")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
#playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbL3J0k32lWnN?si=5c3f6128e25049d9"  #top50DK
playlist_link ="https://open.spotify.com/playlist/4XDFklOPJOJp3geewWo6vo?si=97b4324becef4027"  #Small 6 song playlist
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [5]:
for track in sp.playlist_tracks(playlist_URI)["items"]:
    
    #URI
    track_uri = track["track"]["uri"]
    
    #Track name
    track_name = track["track"]["name"]
    
    #Main Artist
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_info = sp.artist(artist_uri)
    
    #Name, popularity, genre
    artist_name = track["track"]["artists"][0]["name"]
    artist_pop = artist_info["popularity"]
    artist_genres = artist_info["genres"]
    
    #Album
    album = track["track"]["album"]["name"]
    
    #Popularity of the track
    track_pop = track["track"]["popularity"]
    
    #Year
    track_year = track["track"]["album"]["release_date"]
    
    print(track_year)
    #Printin details
    #print(track_name,"\n", 
    #      artist_name,"\n",
    #      #sp.audio_features(track_uri)[0], "\n", 
    #      track_uri
    #     )

2019-06-28
2020-04-03
2021-05-14
2019-07-19
2021-08-16


In [6]:
sp.audio_features("https://open.spotify.com/track/7MXVkk9YMctZqd1Srtv4MB?si=e11b8d5376554fde")[0]

{'danceability': 0.679,
 'energy': 0.587,
 'key': 7,
 'loudness': -7.015,
 'mode': 1,
 'speechiness': 0.276,
 'acousticness': 0.141,
 'instrumentalness': 6.35e-06,
 'liveness': 0.137,
 'valence': 0.486,
 'tempo': 186.003,
 'type': 'audio_features',
 'id': '7MXVkk9YMctZqd1Srtv4MB',
 'uri': 'spotify:track:7MXVkk9YMctZqd1Srtv4MB',
 'track_href': 'https://api.spotify.com/v1/tracks/7MXVkk9YMctZqd1Srtv4MB',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7MXVkk9YMctZqd1Srtv4MB',
 'duration_ms': 230453,
 'time_signature': 4}

In [7]:
data = pd.read_csv("../data/cleanedDatasetWithFeatures.csv")

In [8]:
data.head()

,Unnamed: 0.1,Unnamed: 0,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,1,10123748,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,2,1305825,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,3,2943946,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,4,19131998,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


## Adding features of each song to the dataset

In [9]:
test_set = data.head()

In [10]:
test_set.shape

(5, 20)

In [ ]:
# Add columns to existing dataset with the audio features.
# Implementing a timer to check the time for adding new columns.
import time
tic = time.perf_counter()

errors = 0

for index, row in data[6110:].iterrows():
    try:
        current = sp.audio_features(row['url'])[0]
        data.at[index,'danceability'] = float(current['danceability'])
        data.at[index,'energy'] = current['energy']
        data.at[index,'key'] = current['key']
        data.at[index,'loudness'] = current['loudness']
        data.at[index,'mode'] = current['mode']
        data.at[index,'speechiness'] = current['speechiness']
        data.at[index,'acousticness'] = current['acousticness']
        data.at[index,'instrumentalness'] = current['instrumentalness']
        data.at[index,'liveness'] = current['liveness']
        data.at[index,'valence'] = current['valence']
        data.at[index,'tempo'] = current['tempo']
        data.at[index,'duration_ms'] = current['duration_ms']
        print(str(int(((index+1)/data.shape[0])*100)) + '% done - ' + str(index+1) + ' of ' + str(data.shape[0]) + ' songs iterated - ' + errors + ' encountered.', end='\r')
    except:
        #append error or insert specific values?
        errors = errors+1
        pass
    
        
toc = time.perf_counter()
print(f"Added new rows from external links in {toc - tic:0.4f} seconds")

In [57]:
data.isnull().sum()

Unnamed: 0           0
title                0
rank                 0
date                 0
artist               0
url                  0
region               0
streams              0
danceability        17
energy              17
key                 17
loudness            17
speechiness         17
acousticness        17
instrumentalness    17
liveness            17
valence             17
tempo               17
duration_ms         17
dtype: int64

**Missing values from iteration:**

In [71]:
'''
6119
20101
23710 
25962
33986
34521 
38726 
39988 
52686
63662 
64375
66721
72738
77099 
94502 
96669 
99117 
'''

'\n6119\n20101\n23710 \n25962\n33986\n34521 \n38726 \n39988 \n52686\n63662 \n64375\n66721\n72738\n77099 \n94502 \n96669 \n99117 \n'

In [ ]:
data[6118:6120]

In [58]:
#Saving the sub dataset for testing.
from pathlib import Path
path = Path('../data/cleanedDatasetWithFeatures.csv')
path.parent.mkdir(parents=True, exist_ok=True)
data.to_csv(path)

In [68]:
new_data = data.head()

In [78]:
new_data.head()

,Unnamed: 0,title,rank,date,artist,url,region,streams,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,253019.0,0.852,0.773,8.0,-2.921,0.0776,0.187,0.000030,0.1590,0.9070,102.034,195840.0
1,10123748,Despacito (Featuring Daddy Yankee),1,2017-04-07,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Dominican Republic,11681.0,0.660,0.786,2.0,-4.757,0.1700,0.209,0.000000,0.1120,0.8460,177.833,228200.0
2,1305825,ภาพจำ,1,2018-04-12,Pop Pongkool,https://open.spotify.com/track/4YRpjJaJpiI36HM...,Thailand,15477.0,0.557,0.585,5.0,-7.559,0.0470,0.426,0.000001,0.1440,0.4640,81.922,274000.0
3,2943946,Te Boté - Remix,1,2018-04-27,"Nio Garcia, Casper Magico, Bad Bunny, Darell, ...",https://open.spotify.com/track/3V8UKqhEK5zBkBb...,Honduras,12869.0,0.903,0.675,11.0,-3.445,0.2140,0.542,0.000013,0.0595,0.4420,96.507,417920.0
4,19131998,Binibini,1,2021-04-16,Zack Tabudlo,https://open.spotify.com/track/2X5AFygz5SDYlXa...,Philippines,401678.0,0.642,0.374,5.0,-10.606,0.0320,0.423,0.000002,0.4350,0.0979,129.863,221538.0


In [67]:
new_data.columns[1]

'rank'